In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!sudo apt-get update -y
!sudo apt-get install -y python3.11 python3.11-venv python3.11-dev

In [ ]:
%cd /content/drive/MyDrive/LIT-for-Voice/Backend

In [ ]:
!python3.11 -m venv .venv
!.venv/bin/pip install --upgrade pip
!.venv/bin/pip install -r requirements.txt

In [ ]:
!sudo apt-get install -y redis-server
!redis-server --daemonize yes
!redis-cli ping

In [ ]:
!fuser -k 8000/tcp || true
!pkill -f "uvicorn app.main:app" || true
%env MPLBACKEND=Agg
!nohup .venv/bin/python -m uvicorn app.main:app --host 0.0.0.0 --port 8000 > server.log 2>&1 &

###  *    Set your ngrok token below    *

In [ ]:
!pip -q install pyngrok
from pyngrok import ngrok
# If you have an auth token, uncomment and set it below
ngrok.set_auth_token('')
public_url = ngrok.connect(8000, bind_tls=True).public_url
print('Public URL:', public_url)

In [ ]:
!curl -sS http://127.0.0.1:8000/health || tail -n 100 /content/drive/MyDrive/LIT-for-Voice/Backend/server.log

In [ ]:
import os, subprocess, time, pathlib, psutil

backend_dir = pathlib.Path("/content/drive/MyDrive/LIT-for-Voice/Backend")
log_path = backend_dir / "server.log"

# Kill any stale uvicorn
for p in psutil.process_iter(attrs=["pid","cmdline"]):
    try:
        cmd = " ".join(p.info.get("cmdline") or [])
        if "uvicorn app.main:app" in cmd:
            p.terminate()
    except Exception:
        pass

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["MAX_SALIENCY_SECONDS"] = "12"           # can raise/lower later
os.environ["MAX_SALIENCY_SECONDS_SHAP"] = "6"       # keep low to avoid SHAP spikes
os.environ["SALIENCY_SHAP_SAMPLES"] = "8"           # fewer samples -> less RAM

# Configure CORS origins and cookie settings
os.environ["ALLOWED_ORIGINS"] = f"http://127.0.0.1:8080,http://localhost:8080,{public_url}"
os.environ["COOKIE_SAMESITE"] = "none"
os.environ["COOKIE_SECURE"] = "true"
os.environ["CORS_ALLOW_CREDENTIALS"] = "true"
os.environ["CORS_ALLOW_METHODS"] = "GET, POST, PUT, DELETE, OPTIONS"
os.environ["CORS_ALLOW_HEADERS"] = "Content-Type, Authorization, X-Requested-With"

# Start backend
p = subprocess.Popen(
    [str(backend_dir/".venv/bin/python"), "-m", "uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8000"],
    cwd=str(backend_dir),
    stdout=open(log_path, "ab"),
    stderr=subprocess.STDOUT,
    env=os.environ.copy()
)

time.sleep(2)
print("Backend PID:", p.pid)